In [16]:
import os
from pathlib import Path
import pandas as pd
from IPython.display import display


pd.options.display.max_rows = 10
pd.options.display.max_columns = 1000
pd.set_option("display.max_colwidth", None)


def read_data(path: Path) -> pd.DataFrame:
    return pd.read_json(path)


for i in Path(os.getenv("DATA_DIR")).rglob("*.json"):
    print(i)
    display(read_data(i))

/Users/yo/Developer/repo/sign-to-migrate/data/cpi.json


,countryiso3code,country_name,date,value
0,AFG,Afghanistan,2019,149.895975
1,AFG,Afghanistan,2018,146.522482
2,AFG,Afghanistan,2017,145.610741
3,AFG,Afghanistan,2016,138.708666
4,AFG,Afghanistan,2015,132.883209
...,...,...,...,...
2520,ZWE,Zimbabwe,2014,108.859485
2521,ZWE,Zimbabwe,2013,109.075219
2522,ZWE,Zimbabwe,2012,107.320581
2523,ZWE,Zimbabwe,2011,103.466130


/Users/yo/Developer/repo/sign-to-migrate/data/unemployment_rate.json


,country,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Aruba,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15+",2011,8.90,None,None,None,None
1,Aruba,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15+",2011,8.60,None,None,None,None
2,Aruba,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15+",2011,7.80,None,None,None,None
3,Aruba,PC - Population and Housing Census,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15+",2010,10.60,None,Nonstandard age group: Including age 14,None,Data reference period: September | Age coverage - minimum age: 14 years old
4,Aruba,PC - Population and Housing Census,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Total,"Age (Youth, adults): 15-24",2010,28.86,None,Nonstandard age group: Including age 14,None,Data reference period: September | Age coverage - minimum age: 14 years old
...,...,...,...,...,...,...,...,...,...,...,...
43501,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 15-24",2004,7.13,None,None,None,"Data reference period: June | Geographical coverage: Total national, excluding some areas | Population coverage: Excluding both institutional population and armed forces and/or conscripts"
43502,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Male,"Age (Youth, adults): 25+",2004,2.75,None,None,None,"Data reference period: June | Geographical coverage: Total national, excluding some areas | Population coverage: Excluding both institutional population and armed forces and/or conscripts"
43503,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15+",2004,4.53,None,None,None,"Data reference period: June | Geographical coverage: Total national, excluding some areas | Population coverage: Excluding both institutional population and armed forces and/or conscripts"
43504,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate (%),Sex: Female,"Age (Youth, adults): 15-24",2004,8.13,None,None,None,"Data reference period: June | Geographical coverage: Total national, excluding some areas | Population coverage: Excluding both institutional population and armed forces and/or conscripts"


/Users/yo/Developer/repo/sign-to-migrate/data/women_in_senior_and_middle_position.json


...

,country,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Total,2021,5.679,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
1,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Persons with disability,2021,NaN,Unreliable,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
2,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Total,Disability status (Aggregate): Persons without disability,2021,5.696,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
3,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Total,2021,5.729,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
4,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons with disability,2021,NaN,Unreliable,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
...,...,...,...,...,...,...,...,...,...,...,...
7774,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons with disability,2014,3.356,None,None,None,Repository: ILO-STATISTICS - Micro data processing
7775,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Male,Disability status (Aggregate): Persons without disability,2014,4.531,None,None,None,Repository: ILO-STATISTICS - Micro data processing
7776,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Female,Disability status (Aggregate): Total,2014,5.176,None,None,None,Repository: ILO-STATISTICS - Micro data processing
7777,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.5.2 - Unemployment rate by disability status (%),Sex: Female,Disability status (Aggregate): Persons with disability,2014,2.937,None,None,None,Repository: ILO-STATISTICS - Micro data processing


/Users/yo/Developer/repo/sign-to-migrate/data/informal_employment.json


,country,source,indicator,sex,classif1,time,obs_value,obs_status,note_classif,note_indicator,note_source
0,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Total",2021,86.395,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
1,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Agriculture",2021,99.150,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
2,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Total,"Economic activity (Agriculture, Non-Agriculture): Non-agriculture",2021,74.240,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
3,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Male,"Economic activity (Agriculture, Non-Agriculture): Total",2021,82.987,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
4,Afghanistan,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Male,"Economic activity (Agriculture, Non-Agriculture): Agriculture",2021,98.841,None,None,None,Repository: ILO-STATISTICS - Micro data processing | Data reference period: First quarter
...,...,...,...,...,...,...,...,...,...,...,...
57029,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,Economic activity (Aggregate): Manufacturing,2011,79.230,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data processing
57030,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,Economic activity (Aggregate): Construction,2011,81.350,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data processing
57031,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,"Economic activity (Aggregate): Mining and quarrying; Electricity, gas and water supply",2011,69.238,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data processing
57032,Zimbabwe,LFS - Labour Force Survey,SDG indicator 8.3.1 - Proportion of informal employment in total employment by sex and sector (%),Sex: Female,"Economic activity (Aggregate): Trade, Transportation, Accommodation and Food, and Business and Administrative Services",2011,79.882,Break in series,None,Break in series: Methodology revised,Repository: ILO-STATISTICS - Micro data processing


/Users/yo/Developer/repo/sign-to-migrate/data/happiness.json


,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Year,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Dystopia Residual,Lower Confidence Interval,Upper Confidence Interval,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Trust..Government.Corruption.,Dystopia.Residual
0,1.0,Finland,7.769,1.340,1.587,0.986,0.596,0.153000,0.393,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,Denmark,7.600,1.383,1.573,0.996,0.592,0.252000,0.410,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,Norway,7.554,1.488,1.582,1.028,0.603,0.271000,0.341,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,Iceland,7.494,1.380,1.624,1.026,0.591,0.354000,0.118,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322000,0.298,2019,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
777,NaN,None,NaN,NaN,NaN,NaN,NaN,0.252756,NaN,2017,Rwanda,None,NaN,NaN,NaN,NaN,0.945707,NaN,0.581844,NaN,NaN,NaN,NaN,151.0,3.471,3.543030,3.398970,0.368746,0.326425,0.455220,0.540061
778,NaN,None,NaN,NaN,NaN,NaN,NaN,0.493664,NaN,2017,Syria,None,NaN,NaN,NaN,NaN,0.396103,NaN,0.081539,NaN,NaN,NaN,NaN,152.0,3.462,3.663669,3.260331,0.777153,0.500533,0.151347,1.061574
779,NaN,None,NaN,NaN,NaN,NaN,NaN,0.354256,NaN,2017,Tanzania,None,NaN,NaN,NaN,NaN,1.041990,NaN,0.390018,NaN,NaN,NaN,NaN,153.0,3.349,3.461430,3.236570,0.511136,0.364509,0.066035,0.621130
780,NaN,None,NaN,NaN,NaN,NaN,NaN,0.204435,NaN,2017,Burundi,None,NaN,NaN,NaN,NaN,0.629794,NaN,0.059901,NaN,NaN,NaN,NaN,154.0,2.905,3.074690,2.735310,0.091623,0.151611,0.084148,1.683024
